In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
# 메인 데이터
train = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv", parse_dates=['date'])
test = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv", parse_dates=['date'])

# 보조 데이터
stores = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
holidays = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv", parse_dates=['date'])
transactions = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv", parse_dates=['date'])

In [3]:
for df in [train, test]:
    # 연도 추출
    df['year']=df['date'].dt.year
    # 월 추출
    df['month']=df['date'].dt.month
    # 일 추출
    df['day']=df['date'].dt.day
    # 요일 추출
    df['dayofweek'] = df['date'].dt.dayofweek
    # 주말인지 확인
    df['weekend'] = df['dayofweek'].isin([5,6]).astype(int)

In [4]:
train = train.merge(stores, on="store_nbr", how="left")
test = test.merge(stores, on="store_nbr", how="left")

In [5]:
train = train.merge(transactions, on=["store_nbr","date"], how="left")
test = test.merge(transactions, on=["store_nbr","date"], how="left")

In [6]:
train

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,date,store_nbr,family,sales,onpromotion,year,month,day,dayofweek,weekend,city,state,type,cluster,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,2013,1,1,1,0,Quito,Pichincha,D,13,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,2013,1,1,1,0,Quito,Pichincha,D,13,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,2013,1,1,1,0,Quito,Pichincha,D,13,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,2013,1,1,1,0,Quito,Pichincha,D,13,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,2013,1,1,1,0,Quito,Pichincha,D,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,2017,8,15,1,0,Quito,Pichincha,B,6,2155.0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,2017,8,15,1,0,Quito,Pichincha,B,6,2155.0
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,2017,8,15,1,0,Quito,Pichincha,B,6,2155.0
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017,8,15,1,0,Quito,Pichincha,B,6,2155.0


In [7]:
train['transactions']

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
            ...  
3000883    2155.0
3000884    2155.0
3000885    2155.0
3000886    2155.0
3000887    2155.0
Name: transactions, Length: 3000888, dtype: float64

In [8]:
# 결측치는 0으로 채우기
train['transactions'] = train['transactions'].fillna(0)
test['transactions'] = test['transactions'].fillna(0)

In [9]:
# 공휴일 분류
holidays["is_holiday"] = 1

In [10]:
holidays_sim = holidays [["date", "is_holiday"]].drop_duplicates()

In [11]:
holidays_sim

,date,is_holiday
0,2012-03-02,1
1,2012-04-01,1
2,2012-04-12,1
3,2012-04-14,1
4,2012-04-21,1
...,...,...
344,2017-12-22,1
346,2017-12-23,1
347,2017-12-24,1
348,2017-12-25,1


In [12]:
# 데이터 합치기
train = train.merge(holidays_sim, on="date", how="left")
test = test.merge(holidays_sim, on="date", how="left")

In [13]:
train

,id,date,store_nbr,family,sales,onpromotion,year,month,day,dayofweek,weekend,city,state,type,cluster,transactions,is_holiday
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,2013,1,1,1,0,Quito,Pichincha,D,13,0.0,1.0
1,1,2013-01-01,1,BABY CARE,0.000,0,2013,1,1,1,0,Quito,Pichincha,D,13,0.0,1.0
2,2,2013-01-01,1,BEAUTY,0.000,0,2013,1,1,1,0,Quito,Pichincha,D,13,0.0,1.0
3,3,2013-01-01,1,BEVERAGES,0.000,0,2013,1,1,1,0,Quito,Pichincha,D,13,0.0,1.0
4,4,2013-01-01,1,BOOKS,0.000,0,2013,1,1,1,0,Quito,Pichincha,D,13,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,2017,8,15,1,0,Quito,Pichincha,B,6,2155.0,1.0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,2017,8,15,1,0,Quito,Pichincha,B,6,2155.0,1.0
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,2017,8,15,1,0,Quito,Pichincha,B,6,2155.0,1.0
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,2017,8,15,1,0,Quito,Pichincha,B,6,2155.0,1.0


In [14]:
train['is_holiday'] = train['is_holiday'].fillna(0)
test['is_holiday'] = test['is_holiday'].fillna(0)

In [15]:
# 범주형 데이터 골라내기

cat_cols = train.select_dtypes(include='object').columns.tolist()

In [16]:
train = pd.get_dummies(train, columns=cat_cols)
test = pd.get_dummies(test, columns=cat_cols)

In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 89 columns):
 #   Column                                Dtype         
---  ------                                -----         
 0   id                                    int64         
 1   date                                  datetime64[ns]
 2   store_nbr                             int64         
 3   sales                                 float64       
 4   onpromotion                           int64         
 5   year                                  int32         
 6   month                                 int32         
 7   day                                   int32         
 8   dayofweek                             int32         
 9   weekend                               int64         
 10  cluster                               int64         
 11  transactions                          float64       
 12  is_holiday                            float64       
 13  family_AUTOM

In [18]:
y = train["sales"]

X = train.drop(columns=["id","date", "sales"])
X_test = test.drop(columns=["id","date"])

In [19]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=100,  # 트리 개수
    max_depth=10,      # 트리 최대 깊이
    random_state=42    # 랜덤 고정 (결과 재현 가능하게)
)

In [20]:
model.fit(X, y)

RandomForestRegressor(max_depth=10, random_state=42)

In [21]:
predictions = model.predict(X_test)

In [22]:
# sample_submission 불러오기
submission = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv")

# sales 컬럼에 예측 결과 넣기
submission['sales'] = predictions

# 저장
submission.to_csv('submission.csv', index=False)

In [23]:
submission

,id,sales
0,3000888,28.740364
1,3000889,28.740364
2,3000890,106.258202
3,3000891,3860.584773
4,3000892,28.740364
...,...,...
28507,3029395,106.258202
28508,3029396,28.740364
28509,3029397,4212.601347
28510,3029398,223.793089
